In [1]:
# show current version of this code/project
# sub-module wide the version is tracked using `VERSION` file, and
# code versioning follows https://semver.org/
open("VERSION", 'rt').read() # bump codecov

'v0.0.1-alpha #semver-2.0.0'

<h1 align = "center">CRUD Location Master</h1>

---

The notebook fetches data from freely available source(s) to populate `LocationMaster` schema. The schema represents a set of utility table that stores the *place of origin* of a user. Check more information in `CreateStatement.sql` for individual table information. The dataframe object represents the structure as defined in MySQL database.

**Data Sources** The following data sources are currently integrated:
1. [dr5hn/countries-states-cities-database](https://github.com/dr5hn/countries-states-cities-database)

In [2]:
import yaml

In [3]:
from time import ctime # will be used in logging, file/output directory create etc.
from os import makedirs # create directories dynamically, if not already done so manually
from os.path import join # keep directories `os`-independent
from random import choice # randomly choose from a list of elements
from copy import deepcopy # `pd.Dataframe` is mutable, so any `df` operation may need `deepcopy`
from tqdm import tqdm as TQ # provide progress bar for code completions
from uuid import uuid4 as UUID # keep output file name unique
from string import ascii_uppercase # all ascii upercase letters
from datetime import datetime as dt # formatting datetime objects

In [4]:
# import numpy as np
import pandas as pd

In [5]:
import xlwings as xw # https://www.xlwings.org/

In [6]:
# `today` is so configured that it permits windows/*.nix file/directory names
# also a random hash code of seven charecters is apended to the file name, which
# ensures unique name under each code run. this code saves the output in `data`
today = dt.strftime(dt.strptime(ctime(), "%a %b %d %H:%M:%S %Y"), "%a, %b %d %Y")
random = str(UUID()).upper()[:7]

# output file name is generated here, control the name from here, or
# define a pre-specified name
OUTPUT_FILE = join(".", "data", f"Location Master ({today}) #{random}.xlsx")

print(f"Code Execution Started on: {today}") # only date
print(f"Output File Name: {OUTPUT_FILE}")

Code Execution Started on: Thu, Jun 16 2022
Output File Name: .\data\Location Master (Thu, Jun 16 2022) #350846E.xlsx


In [7]:
IDGenerator = lambda n = 8 : str(UUID().hex[:(n - 1)]).upper() + choice(ascii_uppercase) # generate a `n`-charecter random sequence

## Country Master

Fetch the Country Details, and generate relevant tables.

In [8]:
countries = pd.read_csv(
    "https://github.com/dr5hn/countries-states-cities-database/raw/master/csv/countries.csv",
    usecols = ["name", "iso2", "iso3", "numeric_code", "tld", "currency", "region", "subregion", "currency_name", "currency_symbol", "latitude", "longitude", "capital"],
    encoding = "utf8"
)

# rename columns, to reflect the ones in database
countries.rename(columns = {
    "name" : "CountryName",
    "iso3" : "ISO3",
    "iso2" : "ISO2",
    "numeric_code" : "NumericCode",
    "currency" : "CurrencyCode",
    "currency_name" : "CurrencyName",
    "tld" : "TLDCode",
    "region" : "RegionName",
    "subregion" : "SubRegionName",
    "currency_symbol" : "CurrencySymbol",
    "latitude" : "CountryLat",
    "longitude" : "CountryLon"
}, inplace = True)

countries["CountryLat"] = countries["CountryLat"].apply(lambda x : round(x, 3))
countries["CountryLon"] = countries["CountryLon"].apply(lambda x : round(x, 3))

# generate id columns
countries["CountryID"] = [IDGenerator() for _ in range(countries.shape[0])]
countries.sample()

,CountryName,ISO3,ISO2,NumericCode,capital,CurrencyCode,CurrencyName,CurrencySymbol,TLDCode,RegionName,SubRegionName,CountryLat,CountryLon,CountryID
18,Barbados,BRB,BB,52,Bridgetown,BBD,Barbadian dollar,Bds$,.bb,Americas,Caribbean,13.167,-59.533,27BFF76M


In [9]:
states = pd.read_csv(
    "https://github.com/dr5hn/countries-states-cities-database/raw/master/csv/states.csv",
    usecols = ["name", "country_code", "state_code", "latitude", "longitude"],
    encoding = "utf8"
)

# rename columns, to reflect the ones in database
states.rename(columns = {
    "name" : "StateName",
    "country_code" : "CountryCode",
    "state_code" : "StateCode",
    "latitude" : "StateLat",
    "longitude" : "StateLon"
}, inplace = True)

states["StateLat"] = states["StateLat"].apply(lambda x : round(x, 3))
states["StateLon"] = states["StateLon"].apply(lambda x : round(x, 3))
states["StateCode"] = states["StateCode"].astype(str)

# merge on countries, and get country id
states = states.merge(countries.reset_index()[["CountryID", "ISO2"]], left_on = "CountryCode", right_on = "ISO2", how = "left")

# generate id columns
states["StateID"] = [IDGenerator() for _ in range(states.shape[0])]
states.sample()

,StateName,CountryCode,StateCode,StateLat,StateLon,CountryID,ISO2,StateID
1867,Oristano,IT,OR,40.06,8.748,D8147D8X,IT,9F7343FA


In [10]:
# update capital name with state id
countries = countries.merge(states[["StateID", "StateName"]], left_on = "capital", right_on = "StateName", how = "left")
countries.rename(columns = {"StateID" : "CountryCapital"}, inplace = True)
countries.drop(columns = ["capital", "StateName"], inplace = True)

countries.sample()

,CountryName,ISO3,ISO2,NumericCode,CurrencyCode,CurrencyName,CurrencySymbol,TLDCode,RegionName,SubRegionName,CountryLat,CountryLon,CountryID,CountryCapital
144,Micronesia,FSM,FM,583,USD,United States dollar,$,.fm,Oceania,Micronesia,6.917,158.25,577A9D4Y,NaN


In [11]:
# get currency dataset
currency = countries[["CurrencyCode", "CurrencyName", "CurrencySymbol"]].dropna().drop_duplicates()

# drop duplicate currency name
currency.drop_duplicates("CurrencyCode", inplace = True)
currency.drop_duplicates(["CurrencyCode", "CurrencyName"], inplace = True)

currency.sample()

,CurrencyCode,CurrencyName,CurrencySymbol
30,NOK,Norwegian Krone,kr


In [12]:
# remove processed columns from `countries`
countries.drop(columns = ["CurrencyName", "CurrencySymbol"], inplace = True)
countries.sample()

,CountryName,ISO3,ISO2,NumericCode,CurrencyCode,TLDCode,RegionName,SubRegionName,CountryLat,CountryLon,CountryID,CountryCapital
192,Saint-Barthelemy,BLM,BL,652,EUR,.bl,Americas,Caribbean,18.5,-63.417,53E634FJ,NaN


In [13]:
# get a set of unique regions
regions = countries[["RegionName", "SubRegionName"]].dropna().drop_duplicates().sort_values(["RegionName", "SubRegionName"])

# generate unique ids
regions["RegionID"] = [IDGenerator() for _ in range(regions.shape[0])]
regions.sample()

,RegionName,SubRegionName,RegionID
21,Americas,Central America,2B254B7I


In [14]:
countries = countries.merge(regions, on = ["RegionName", "SubRegionName"], how = "left")

# remove processed columns from `countries`
countries.drop(columns = ["RegionName", "SubRegionName"], inplace = True)
countries.sample()

,CountryName,ISO3,ISO2,NumericCode,CurrencyCode,TLDCode,CountryLat,CountryLon,CountryID,CountryCapital,RegionID
55,Croatia,HRV,HR,191,HRK,.hr,45.167,15.5,BC8FCD0S,A67EE6DY,08F653CG


## Save to Disk

In [15]:
states.drop_duplicates(["StateID", "StateName"], inplace = True)
countries.drop_duplicates(["CountryID", "CountryName"], inplace = True)

In [16]:
countries.set_index("CountryID", inplace = True)
states.set_index("StateID", inplace = True)
currency.set_index("CurrencyCode", inplace = True)
regions.set_index("RegionID", inplace = True)

In [17]:
regions.sample()

,RegionName,SubRegionName
RegionID,,
08F653CG,Europe,Southern Europe


In [18]:
countries = countries[["CountryName", "ISO3", "ISO2", "NumericCode", "RegionID", "CurrencyCode", "TLDCode", "CountryLat", "CountryLon", "CountryCapital"]]
countries.sample()

,CountryName,ISO3,ISO2,NumericCode,RegionID,CurrencyCode,TLDCode,CountryLat,CountryLon,CountryCapital
CountryID,,,,,,,,,,
A7E507FU,Dominica,DMA,DM,212,A22C9E2G,XCD,.dm,15.417,-61.333,NaN


In [19]:
states = states[["StateName", "StateCode", "CountryID", "StateLat", "StateLon"]]
states.sample()

,StateName,StateCode,CountryID,StateLat,StateLon
StateID,,,,,
3BBAAE5K,Inverclyde,IVC,81866AFA,55.932,-4.68


In [20]:
currency.sample()

,CurrencyName,CurrencySymbol
CurrencyCode,,
AFN,Afghan afghani,؋


In [21]:
wb = xw.Book(join(".", "data", r"Location Master Template.xlsx"))

# about sheet is populated from `CodeMeta.yml` file
with open(join(".", "data", "CodeMeta.yaml")) as f:
    meta = yaml.load(f, Loader = yaml.FullLoader)
    
for cell, value in meta["about"].items():
    if cell == "J7":
        value = eval(value) # check comment on meta

    wb.sheets["about"][cell].value = value
    
wb.sheets["about"]["C11"].value = ctime()

# update/add dataframe to all other sheets
wb.sheets["RegionMaster"]["A1"].value = regions
wb.sheets["CountryMaster"]["A1"].value = countries
wb.sheets["StateMaster"]["A1"].value = states
wb.sheets["CurrencyMaster"]["A1"].value = currency

# save to a new file, with the convention specified above
wb.save(OUTPUT_FILE); wb.close()

```python
python main.py localhost 33060 root admin179 LocationMaster "./data/Location Master (Thu, Jun 16 2022) #716A6AE.xlsx"
```